# Método para solução de sistemas lineares do tipo "Kd=F"

## Solução de sistemas invertendo a Matriz K

### Feito por: Leonardo Minski

In [18]:
import numpy as np


In [19]:
E = 3000 #kN/cmˆ2
bw = 15 # cm
h = 40 # cm
I = bw*h**3/12# cmˆ4
L = 100 # cm
P = 50 #kN

Ke = np.array([[12*E*I/L**3, 6*E*I/L**2, -12*E*I/L**3, 6*E*I/L**2],
      [6*E*I/L**2,  4*E*I/L,    -6*E*I/L**2,  2*E*I/L],
      [-12*E*I/L**3, -6*E*I/L**2, 12*E*I/L**3, -6*E*I/L**2],
      [6*E*I/L**2, 2*E*I/L, -6*E*I/L**2, 4*E*I/L]])

print(Ke)

[[ 2.88e+03  1.44e+05 -2.88e+03  1.44e+05]
 [ 1.44e+05  9.60e+06 -1.44e+05  4.80e+06]
 [-2.88e+03 -1.44e+05  2.88e+03 -1.44e+05]
 [ 1.44e+05  4.80e+06 -1.44e+05  9.60e+06]]


In [20]:
#Número de nós é igual a 3 e o número de graus de liberdade é igual a 6
Num_Barras = 2
Numero_Nos = Num_Barras + 1 
Graus_Liberdade = Num_Barras * 2 + 2 # Graus de liberdade, no caso é deslocamento, rotação, deslocamento, rotação e assim por diante...
Matriz_geral = [] # Matriz principal onde vão ser adicionados os valores
for i in range(Graus_Liberdade): # Matriz quadrada
    Matriz_geral.append(np.zeros(Graus_Liberdade))

Matriz_geral # Matriz de rigidez geral

[array([0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0.])]

In [21]:
#Verificar quantidade de barras
#Somar na matriz os termos considerando o termo inicial + 3 termos a direita na linha atual, depois + 3 linhas para baixo a partir do i e do j atual
for termo_inicial in range(0, Num_Barras * 2, 2):
    for coluna in range(4):
        coluna_matriz_global = termo_inicial + coluna
        for linha in range(4): 
           Matriz_geral[coluna_matriz_global][termo_inicial+linha] += Ke[coluna][linha]

Matriz_geral # Matriz de rigidez geral a partir do número de barras, se alterar o numero de barras na célula acima, é possível ver como que ela fica com maiores números de barras.

[array([  2880., 144000.,  -2880., 144000.,      0.,      0.]),
 array([ 144000., 9600000., -144000., 4800000.,       0.,       0.]),
 array([  -2880., -144000.,    5760.,       0.,   -2880.,  144000.]),
 array([  144000.,  4800000.,        0., 19200000.,  -144000.,  4800000.]),
 array([      0.,       0.,   -2880., -144000.,    2880., -144000.]),
 array([      0.,       0.,  144000., 4800000., -144000., 9600000.])]

In [22]:
# Como sabemos que é uma viga biapoiada e os deslocamentos nos apoios são iguais a zero, o nosso vetor de deslocamentos é algo assim:

#Como não temos uma interface para identificar os apoios, temos que definir em uma matriz, como sabemos que é uma viga biapoiada com duas barras, é pra ser algo assim:

Deslocamento_Desconhecido = None
Vetor_Deslocamentos = [0,
                      Deslocamento_Desconhecido,
                      Deslocamento_Desconhecido,
                      Deslocamento_Desconhecido,
                      0,
                      Deslocamento_Desconhecido]

In [23]:
indices_remover = []  # Vamos armazenar os índices dos termos a serem removidos

# Encontrar os índices dos termos iguais a zero
for i, termo in enumerate(Vetor_Deslocamentos):
    if termo == 0:
        indices_remover.append(i)

indices_remover.sort(reverse=True) # invertendo para não remover termos errados

Matriz_geral = np.array(Matriz_geral)
Matriz_geral = np.delete(Matriz_geral, indices_remover, axis=0) # Removendo linha
Matriz_geral = np.delete(Matriz_geral, indices_remover, axis=1) # Removendo coluna

Matriz_geral

array([[ 9.60e+06, -1.44e+05,  4.80e+06,  0.00e+00],
       [-1.44e+05,  5.76e+03,  0.00e+00,  1.44e+05],
       [ 4.80e+06,  0.00e+00,  1.92e+07,  4.80e+06],
       [ 0.00e+00,  1.44e+05,  4.80e+06,  9.60e+06]])

In [24]:
Matriz_geral_invertida = np.linalg.inv(Matriz_geral) # K elevado a -1, inversão de matriz.

Matriz_geral_invertida

array([[ 2.77777778e-07,  1.04166667e-05, -3.47222222e-08,
        -1.38888889e-07],
       [ 1.04166667e-05,  6.94444444e-04,  0.00000000e+00,
        -1.04166667e-05],
       [-3.47222222e-08,  0.00000000e+00,  6.94444444e-08,
        -3.47222222e-08],
       [-1.38888889e-07, -1.04166667e-05, -3.47222222e-08,
         2.77777778e-07]])

In [25]:
F = [0, -50, 0, 0]

# Multiplicar cada linha da matriz pelo vetor
Deslocamentos = [np.dot(linha, F) for linha in Matriz_geral_invertida]

Deslocamentos

[-0.0005208333333333332, -0.03472222222222222, 0.0, 0.0005208333333333332]